<a href="https://colab.research.google.com/github/Krupa-1010/Deep_Learning/blob/main/Algae_Detection_Dummy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [42]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:",device)

Using device: cuda


In [43]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [44]:
train_dataset=datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_datset=datasets.CIFAR10(root="./data",train=False,download=True,transform=transform)

In [45]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [46]:
test_datset

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [47]:
train_dataset[0]

(tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
          [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
          [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
          ...,
          [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
          [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
          [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],
 
         [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
          [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
          [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
          ...,
          [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
          [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
          [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],
 
         [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
          [-0.8431, -1.0000,

In [48]:
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=2)
test_loader=DataLoader(test_datset,batch_size=64,shuffle=False,num_workers=2)

In [49]:
#CNN Model
class AlgaeCNN(nn.Module):

  def __init__(self):
    super().__init__()
    self.conv_layers=nn.Sequential(
        nn.Conv2d(3,32,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(32,64,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(64,128,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
      )

    self.fc_layer=nn.Sequential(
        nn.Flatten(),
        nn.Linear(128*4*4,256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256,10),
    )

  def forward(self,x):
    x=self.conv_layers(x)
    x=self.fc_layer(x)
    return x


In [50]:

model=AlgaeCNN().to(device)

In [51]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)


In [52]:
#Train
epochs=20
train_losses=[]
train_accuracy=[]
test_accuracy=[]
for epoch in range(epochs):
  running_loss=0.0
  correct=0
  total=0

  model.train()
  for i,(inputs,labels) in enumerate(train_loader):
    inputs,labels=inputs.to(device),labels.to(device)
    outputs=model(inputs)
    loss=criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    running_loss+=loss.item()

    _,predicted=outputs.max(1)
    total+=labels.size(0)
    correct +=predicted.eq(labels).sum().item()

  average_loss=running_loss/len(train_loader)
  train_losses.append(average_loss)
  tr_accuracy=100*correct /total
  train_accuracy.append(tr_accuracy)

  print(f"Epoch {epoch}, Loss: {average_loss:.4f} Train Accuracy ={tr_accuracy:.2f}%")

  #test

  with torch.no_grad():
    model.eval()
    test_correct=0
    test_total=0
    for b,(test_inputs,test_labels) in enumerate(test_loader):
      test_inputs,test_labels=test_inputs.to(device),test_labels.to(device)
      test_outputs=model(test_inputs)
      _,y_predicted=test_outputs.max(1)
      test_correct+=y_predicted.eq(test_labels).sum().item()
      test_total+=test_labels.size(0)

    te_accuracy=100*test_correct/test_total
    test_accuracy.append(te_accuracy)

    print(f"Test Accuracy:{te_accuracy:.2f}%")




Epoch 0, Loss: 1.4947 Train Accuracy =45.91%
Test Accuracy:58.82%
Epoch 1, Loss: 1.0861 Train Accuracy =61.75%
Test Accuracy:64.60%
Epoch 2, Loss: 0.9040 Train Accuracy =68.26%
Test Accuracy:70.49%
Epoch 3, Loss: 0.7880 Train Accuracy =72.23%
Test Accuracy:72.00%
Epoch 4, Loss: 0.7005 Train Accuracy =75.53%
Test Accuracy:72.59%
Epoch 5, Loss: 0.6382 Train Accuracy =77.59%
Test Accuracy:75.95%
Epoch 6, Loss: 0.5796 Train Accuracy =79.65%
Test Accuracy:76.34%
Epoch 7, Loss: 0.5243 Train Accuracy =81.51%
Test Accuracy:76.44%
Epoch 8, Loss: 0.4783 Train Accuracy =83.21%
Test Accuracy:76.77%
Epoch 9, Loss: 0.4370 Train Accuracy =84.49%
Test Accuracy:76.93%
Epoch 10, Loss: 0.3964 Train Accuracy =86.01%
Test Accuracy:76.83%
Epoch 11, Loss: 0.3690 Train Accuracy =86.60%
Test Accuracy:77.42%
Epoch 12, Loss: 0.3404 Train Accuracy =87.70%
Test Accuracy:76.81%
Epoch 13, Loss: 0.3169 Train Accuracy =88.46%
Test Accuracy:76.73%
Epoch 14, Loss: 0.2955 Train Accuracy =89.17%
Test Accuracy:77.65%
Epoch